OKAY SO LETS GRAB SOME SLICES OF
- METER READINGS
- CUSTOMER DATA
- MARGINAL EMISSIONS

Do  A Join so I have

Customer ID, Timestamp, Usage, Marginal Emissions


# irp-dbk24 - "Optimising Demand Response Strategies for Carbon-Intelligent Electricity Use"

# Joining Data

**NOTEBOOK PURPOSE(S):**

**LIMITATIONS:**

**NOTEBOOK OUTPUTS:**



### Importing Libraries

In [2]:
# ────────────────────────────────────────────────────────────────────────────
# Future (must be first)
# ────────────────────────────────────────────────────────────────────────────
from __future__ import annotations

# ────────────────────────────────────────────────────────────────────────────
# Jupyter/Notebook Setup
# ────────────────────────────────────────────────────────────────────────────
%matplotlib inline
from IPython.display import display

# ────────────────────────────────────────────────────────────────────────────
# Standard Library
# ────────────────────────────────────────────────────────────────────────────
import binascii
import calendar
import json
import logging
import math
import os
import random
import re
import hashlib
import inspect
import time
from copy import deepcopy
from contextlib import contextmanager
from dataclasses import dataclass
from datetime import datetime, timedelta
from functools import partial, wraps
from itertools import combinations, product
from multiprocessing import Manager, Pool, Lock, cpu_count
from multiprocessing.pool import ThreadPool
from pathlib import Path
from typing import (
    Any, Callable, Dict, Iterable, List, Mapping, Optional, Sequence, Tuple, Union
)
from zoneinfo import ZoneInfo

# ────────────────────────────────────────────────────────────────────────────
# Core Data Handling
# ────────────────────────────────────────────────────────────────────────────
import numpy as np
import pandas as pd
import polars as pl

# ────────────────────────────────────────────────────────────────────────────
# Machine Learning & Statistics
# ────────────────────────────────────────────────────────────────────────────
from feature_engine.creation import CyclicalFeatures
from scipy.stats import kurtosis, skew, zscore
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.compose import ColumnTransformer
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    root_mean_squared_error,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, SplineTransformer
from sklearn.utils.validation import check_is_fitted

# ────────────────────────────────────────────────────────────────────────────
# Visualization
# ────────────────────────────────────────────────────────────────────────────
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import seaborn as sns

# ────────────────────────────────────────────────────────────────────────────
# Geospatial
# ────────────────────────────────────────────────────────────────────────────
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.wkb import loads
from pyproj import Proj, transform


## Functions

In [ ]:
def wkb_to_coords(hex_wkb: str):
    """
    Convert a hex WKB string to coordinates (x, y).
    Uses binascii to decode the hex string and shapely's load function to convert WKB to a Point object.

    Parameters:
    ----------
    hex_wkb : str
        Hexadecimal string representing the WKB (Well-Known Binary) format of a point.

    Returns:
    -------
    tuple
        A tuple containing the x and y coordinates of the point, or (None, None)
        if the conversion fails.
    """

    try:
        point = loads(binascii.unhexlify(hex_wkb))
        return point.x, point.y
    except Exception:
        return None, None

## Loading Data

### Directories

In [4]:
# DIRECTORIES AND PATHS
base_data_directory = "data"    # Base directory where the dataframes will be saved
hitachi_data_directory = os.path.join(base_data_directory, "hitachi_copy")      # Directory where the dataframes will be saved
meter_save_directory = os.path.join(hitachi_data_directory, "meter_primary_files")       # Directory for meter readings

marginal_emissions_development_directory = os.path.join(base_data_directory, "marginal_emissions_development")  # Directory for marginal emissions development data
marginal_emissions_results_directory = os.path.join(marginal_emissions_development_directory, "results")
marginal_emissions_logs_directory = os.path.join(marginal_emissions_development_directory, "logs")

optimisation_development_directory = os.path.join(base_data_directory, "optimisation_development")


In [4]:
print("\n" + "-" * 120)
print(f"Contents of '{hitachi_data_directory}' and subdirectories:\n" + "-" * 120)
for root, dirs, files in os.walk(hitachi_data_directory):
    for f in sorted(files):
        rel_dir = os.path.relpath(root, hitachi_data_directory)
        rel_file = os.path.join(rel_dir, f) if rel_dir != "." else f
        print(f"  - {rel_file}")


------------------------------------------------------------------------------------------------------------------------
Contents of 'data/hitachi_copy' and subdirectories:
------------------------------------------------------------------------------------------------------------------------
  - .DS_Store
  - customers_20250714_1401.parquet
  - grid_readings_20250714_1401.parquet
  - grid_readings_20250714_1401_processed.parquet
  - grid_readings_20250714_1401_processed_half_hourly.parquet
  - weather_20250714_1401.parquet
  - weather_20250714_1401_processed.parquet
  - weather_and_grid_data_half-hourly_20250714_1401.parquet
  - weather_data_combined_20250714_1401.parquet
  - meter_primary_files/.DS_Store
  - meter_primary_files/meter_readings_2021_20250714_2015.parquet
  - meter_primary_files/meter_readings_2021_20250714_2015_formatted.parquet
  - meter_primary_files/meter_readings_2021_Q4_20250714_2015_formatted.parquet
  - meter_primary_files/meter_readings_2022_20250714_2324.parq

In [5]:
print("\n" + "-" * 120)
print(f"Contents of '{marginal_emissions_development_directory}' and subdirectories:\n" + "-" * 120)
for root, dirs, files in os.walk(marginal_emissions_development_directory):
    for f in sorted(files):
        rel_dir = os.path.relpath(root, marginal_emissions_development_directory)
        rel_file = os.path.join(rel_dir, f) if rel_dir != "." else f
        print(f"  - {rel_file}")


------------------------------------------------------------------------------------------------------------------------
Contents of 'data/marginal_emissions_development' and subdirectories:
------------------------------------------------------------------------------------------------------------------------
  - .DS_Store
  - marginal_emissions_estimation_20250714_1401_test_data.parquet
  - marginal_emissions_estimation_20250714_1401_train_data.parquet
  - marginal_emissions_estimation_20250714_1401_validation_data.parquet
  - qgam_smooth_weather_tuning_results.csv
  - qgam_smooth_weather_tuning_results2.csv
  - results/.DS_Store
  - results/original_quantile_bins_marginal_emissions_timeseries.parquet
  - results/pyGAM_marginal_emissions_timeseries.parquet
  - logs/marginal_emissions_results.part000.csv
  - logs/marginal_emissions_results_index.csv
  - logs/marginal_emissions_results_median.part000.csv
  - logs/marginal_emissions_results_median_index.csv
  - logs/marginal_emissions_

### File Paths

In [6]:
# Defining Full files
full_customers_filename = "customers_20250714_1401"
full_meter_readings_filename = "meter_readings_all_years_20250714_formatted"
full_marginal_emissions_filename = "original_quantile_bins_marginal_emissions_timeseries"
full_average_emissions_filename = "grid_readings_20250714_1401_processed_half_hourly"

pygam_marginal_emissions_filename = "pyGAM_marginal_emissions_timeseries"

In [7]:
# Defining Full Filepaths
full_customers_filepath = os.path.join(hitachi_data_directory, full_customers_filename + ".parquet")
full_meter_readings_filepath = os.path.join(meter_save_directory, full_meter_readings_filename + ".parquet")
full_marginal_emissions_filepath = os.path.join(marginal_emissions_results_directory, full_marginal_emissions_filename + ".parquet")
full_average_emissions_filepath = os.path.join(hitachi_data_directory, full_average_emissions_filename + ".parquet")

pygam_marginal_emissions_filepath = os.path.join(marginal_emissions_results_directory, pygam_marginal_emissions_filename + ".parquet")

### Getting Snippets of the Data

In [8]:
test_date_range_start = datetime(2022, 5, 4, tzinfo=ZoneInfo("Asia/Kolkata"))
test_date_range_end = datetime(2022, 5, 18, tzinfo=ZoneInfo("Asia/Kolkata"))


str(test_date_range_start)[:10]
str(test_date_range_end)[:10]

'2022-05-18'

#### Average Emissions

In [9]:
average_emissions_pldf = pl.read_parquet(full_average_emissions_filepath)

In [10]:
# inspect data
average_emissions_pldf.describe()

statistic,timestamp,thermal_generation,gas_generation,hydro_generation,nuclear_generation,renewable_generation,total_generation,demand_met,non_renewable_generation,tons_co2,g_co2_per_kwh,tons_co2_per_mwh
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""95088""",95088.0,95088.0,95088.0,95088.0,95088.0,95088.0,95088.0,95088.0,95088.0,95088.0,95088.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""","""2022-09-17 12:15:00""",127346.99741,3880.785326,17808.916682,5098.942404,18015.883059,172146.160136,170937.702059,154135.641823,62916.633837,734.108622,0.734109
"""std""",null,20916.196838,1619.355378,8617.821113,788.865947,14612.596141,26379.403551,26081.970354,22092.847367,10139.404973,76.549608,0.07655
"""min""","""2020-01-01 00:30:00""",66097.833333,109.483135,2516.0,2281.083333,0.0,95950.333333,96848.5,86763.333333,33384.3019,483.357945,0.483358
"""25%""","""2021-05-10 06:30:00""",111923.333333,2565.083333,10134.916667,4444.0,6740.833333,153551.166667,152501.5,138526.0,55405.58295,681.281762,0.681282
"""50%""","""2022-09-17 12:30:00""",128648.583333,3457.833333,16946.166667,5126.666667,12830.5,172655.166667,171276.583333,154052.0,63403.42905,734.684328,0.734684
"""75%""","""2024-01-25 18:00:00""",143295.5,4922.083333,24823.166667,5580.25,25190.333333,192577.333333,191039.902778,169530.333333,70569.09085,797.078569,0.797079
"""max""","""2025-06-04 00:00:00""",177508.083333,12915.75,43162.083333,7070.0,81927.666667,251763.666667,249591.916667,217778.833333,88742.75695,905.09159,0.905092


In [11]:
# Extract start and end dates
start_date_boundary = str(average_emissions_pldf["timestamp"].min())[:10]
end_date_boundary = str(average_emissions_pldf["timestamp"].max())[:10]


In [12]:
print(start_date_boundary)
print(end_date_boundary)

2020-01-01
2025-06-04


In [13]:
# drop everything but emission factor and timestamp
reduced_average_emissions_pldf = average_emissions_pldf.drop(["thermal_generation", "gas_generation","hydro_generation",
                                                      "nuclear_generation","renewable_generation", "total_generation",
                                                      "demand_met", "non_renewable_generation", "tons_co2",
                                                      "tons_co2_per_mwh"])
# check results:
display(reduced_average_emissions_pldf.schema)

Schema([('timestamp', Datetime(time_unit='us', time_zone=None)),
        ('g_co2_per_kwh', Float64)])

In [14]:
reduced_average_emissions_pldf = reduced_average_emissions_pldf.rename({"g_co2_per_kwh": "average_emissions_grams_co2_per_kWh"})

# check results:
display(reduced_average_emissions_pldf.schema)

Schema([('timestamp', Datetime(time_unit='us', time_zone=None)),
        ('average_emissions_grams_co2_per_kWh', Float64)])

In [15]:
# Change the timestamp to be the local timezone
# note we do replace not convert
reduced_average_emissions_pldf = reduced_average_emissions_pldf.with_columns(
    pl.col("timestamp").dt.replace_time_zone("Asia/Kolkata")
)
# check results:
display(reduced_average_emissions_pldf.schema)
display(reduced_average_emissions_pldf.describe())

Schema([('timestamp', Datetime(time_unit='us', time_zone='Asia/Kolkata')),
        ('average_emissions_grams_co2_per_kWh', Float64)])

statistic,timestamp,average_emissions_grams_co2_per_kWh
str,str,f64
"""count""","""95088""",95088.0
"""null_count""","""0""",0.0
"""mean""","""2022-09-17 12:15:00+05:30""",734.108622
"""std""",null,76.549608
"""min""","""2020-01-01 00:30:00+05:30""",483.357945
"""25%""","""2021-05-10 06:30:00+05:30""",681.281762
"""50%""","""2022-09-17 12:30:00+05:30""",734.684328
"""75%""","""2024-01-25 18:00:00+05:30""",797.078569
"""max""","""2025-06-04 00:00:00+05:30""",905.09159


In [16]:
print("Time Boundaries before reduction")
print(f"Minimum: {((reduced_average_emissions_pldf.select('timestamp')).min())}, Maximum: {(reduced_average_emissions_pldf.select('timestamp')).max()}")

# Trim the timeframe to fit the analysis that we are doing:
trimmed_average_emissions_pldf = reduced_average_emissions_pldf.filter(
        (pl.col("timestamp") >= test_date_range_start) & (pl.col("timestamp") <= test_date_range_end)
)
# check results:
print("Time Boundaries after reduction")
print(f"Minimum: {trimmed_average_emissions_pldf.select('timestamp').min()}, Maximum: {trimmed_average_emissions_pldf.select('timestamp').max()}")


Time Boundaries before reduction
Minimum: shape: (1, 1)
┌────────────────────────────┐
│ timestamp                  │
│ ---                        │
│ datetime[μs, Asia/Kolkata] │
╞════════════════════════════╡
│ 2020-01-01 00:30:00 IST    │
└────────────────────────────┘, Maximum: shape: (1, 1)
┌────────────────────────────┐
│ timestamp                  │
│ ---                        │
│ datetime[μs, Asia/Kolkata] │
╞════════════════════════════╡
│ 2025-06-04 00:00:00 IST    │
└────────────────────────────┘
Time Boundaries after reduction
Minimum: shape: (1, 1)
┌────────────────────────────┐
│ timestamp                  │
│ ---                        │
│ datetime[μs, Asia/Kolkata] │
╞════════════════════════════╡
│ 2022-05-04 00:00:00 IST    │
└────────────────────────────┘, Maximum: shape: (1, 1)
┌────────────────────────────┐
│ timestamp                  │
│ ---                        │
│ datetime[μs, Asia/Kolkata] │
╞════════════════════════════╡
│ 2022-05-18 00:00:00 IST    │
└───

In [17]:
trimmed_average_emissions_filename = f"average_emissions_{str(test_date_range_start)[:10]}_to_{str(test_date_range_end)[:10]}"
trimmed_average_emissions_filepath = os.path.join(optimisation_development_directory, trimmed_average_emissions_filename + ".parquet")

In [18]:
try:
    trimmed_average_emissions_pldf.write_parquet(file=trimmed_average_emissions_filepath,
                                                  compression="snappy",
                                                  statistics=True)
    print(f"Successfully saved trimmed average emissions data to {trimmed_average_emissions_filepath}")

except Exception as e:
    print(f"Error occurred while saving trimmed average emissions data: {e}")

Successfully saved trimmed average emissions data to data/optimisation_development/average_emissions_2022-05-04_to_2022-05-18.parquet


#### Marginal Emissions

In [19]:
marginal_emissions_pldf = pl.read_parquet(full_marginal_emissions_filepath)
pygam_marginal_emissions = pl.read_parquet(pygam_marginal_emissions_filepath)

In [20]:
marginal_emissions_pldf.describe()

statistic,timestamp,city,land_longitude,land_latitude,ME,alpha1,alpha2,original_quantile_group_id,demand_met
str,str,str,f64,f64,f64,f64,f64,f64,f64
"""count""","""3481920""","""3481920""",3.48192e6,3.48192e6,3.48192e6,3.48192e6,3.48192e6,3.48192e6,3.48192e6
"""null_count""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""","""2023-03-17 23:45:00+05:30""",null,75.701111,25.393333,0.373358,0.887762,-0.000001,16.530667,177287.744297
"""std""",null,null,1.969453,4.538596,0.073292,0.112286,3.4511e-7,11.399828,23535.203493
"""min""","""2021-01-01 00:00:00+05:30""",null,72.77,18.5,0.101938,0.682504,-0.000002,2.0,105140.916667
"""25%""","""2022-02-08 00:00:00+05:30""",null,72.97,19.2,0.326648,0.798551,-0.000002,4.0,160999.75
"""50%""","""2023-03-18 00:00:00+05:30""",null,76.94,28.5,0.373625,0.869218,-0.000001,17.0,178227.333325
"""75%""","""2024-04-23 23:30:00+05:30""",null,77.14,28.7,0.423545,1.009042,-0.000001,26.0,194806.833333
"""max""","""2025-05-31 23:30:00+05:30""",null,77.34,28.8,0.635273,1.06163,-8.1658e-7,34.0,249591.916667


In [21]:
marginal_emissions_pldf = marginal_emissions_pldf.with_columns(
    # clarifying units - ME currently: tons CO2 per MW per interval
    pl.col("ME").alias("quantile_marginal_emissions_tons_co2_per_MW"),
     # Convert to g CO2 per MWh: divide by interval_hours
    ((pl.col("ME")/0.5)*1000).alias("quantile_marginal_emissions_grams_co2_per_kWh"),
    # clarify units of demand_met
    pl.col("demand_met").alias("demand_met_MW"),
    # convert to kWh
    ((pl.col("demand_met") * 0.5 * 1000).alias("demand_met_kWh"))
)
marginal_emissions_pldf.describe()

statistic,timestamp,city,land_longitude,land_latitude,ME,alpha1,alpha2,original_quantile_group_id,demand_met,quantile_marginal_emissions_tons_co2_per_MW,quantile_marginal_emissions_grams_co2_per_kWh,demand_met_MW,demand_met_kWh
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""3481920""","""3481920""",3.48192e6,3.48192e6,3.48192e6,3.48192e6,3.48192e6,3.48192e6,3.48192e6,3.48192e6,3.48192e6,3.48192e6,3.48192e6
"""null_count""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""","""2023-03-17 23:45:00+05:30""",null,75.701111,25.393333,0.373358,0.887762,-0.000001,16.530667,177287.744297,0.373358,746.715198,177287.744297,8.8644e7
"""std""",null,null,1.969453,4.538596,0.073292,0.112286,3.4511e-7,11.399828,23535.203493,0.073292,146.583329,23535.203493,1.1768e7
"""min""","""2021-01-01 00:00:00+05:30""",null,72.77,18.5,0.101938,0.682504,-0.000002,2.0,105140.916667,0.101938,203.876737,105140.916667,5.2570e7
"""25%""","""2022-02-08 00:00:00+05:30""",null,72.97,19.2,0.326648,0.798551,-0.000002,4.0,160999.75,0.326648,653.296841,160999.75,8.0499875e7
"""50%""","""2023-03-18 00:00:00+05:30""",null,76.94,28.5,0.373625,0.869218,-0.000001,17.0,178227.333325,0.373625,747.250521,178227.333325,8.9114e7
"""75%""","""2024-04-23 23:30:00+05:30""",null,77.14,28.7,0.423545,1.009042,-0.000001,26.0,194806.833333,0.423545,847.089125,194806.833333,9.7403e7
"""max""","""2025-05-31 23:30:00+05:30""",null,77.34,28.8,0.635273,1.06163,-8.1658e-7,34.0,249591.916667,0.635273,1270.545447,249591.916667,1.2480e8


In [22]:
pygam_marginal_emissions = pygam_marginal_emissions.with_columns(
    # clarifying units - ME currently: tons CO2 per MW per interval
    pl.col("ME").alias("pg_marginal_emissions_tons_co2_per_MW"),
     # Convert to g CO2 per MWh: divide by interval_hours
    ((pl.col("ME")/0.5)*1000).alias("pg_marginal_emissions_grams_co2_per_kWh"),
    # clarify units of demand_met
    pl.col("demand_met").alias("demand_met_MW"),
    # convert to kWh
    ((pl.col("demand_met") * 0.5 * 1000).alias("demand_met_kWh")),
    pl.col("ME_cal").alias("pg_marginal_emissions_tons_co2_per_MW_calibrated"),
    (pl.col("ME_cal")/0.5*1000).alias("pg_marginal_emissions_grams_co2_per_kWh_calibrated")
).drop(["y_true", "y_pred","demand_met"])
pygam_marginal_emissions.describe()

statistic,timestamp,land_latitude,land_longitude,ME,city,demand_minus_renewables,National MW Shift,Pearson R Score,Spearman Score,Confidence Level,ME_cal,pg_marginal_emissions_tons_co2_per_MW,pg_marginal_emissions_grams_co2_per_kWh,demand_met_MW,demand_met_kWh,pg_marginal_emissions_tons_co2_per_MW_calibrated,pg_marginal_emissions_grams_co2_per_kWh_calibrated
str,str,f64,f64,f64,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64
"""count""","""3481920""",3.48192e6,3.48192e6,3.48192e6,"""3481920""",3.48192e6,3.48192e6,3.48192e6,3.48192e6,"""3481920""",3.48192e6,3.48192e6,3.48192e6,3.48192e6,3.48192e6,3.48192e6,3.48192e6
"""null_count""","""0""",0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""","""2023-03-17 23:45:00+05:30""",25.393333,75.701111,0.42054,null,157950.088079,2377.697555,0.490469,0.550644,null,0.353163,0.42054,841.079038,177287.744297,8.8644e7,0.353163,706.326047
"""std""",null,4.538596,1.969453,0.032109,null,19898.648364,1890.42524,0.08124,0.049947,null,0.071478,0.032109,64.218552,23535.203493,1.1768e7,0.071478,142.956919
"""min""","""2021-01-01 00:00:00+05:30""",18.5,72.77,0.331015,null,95019.083333,0.0,0.283472,0.448289,"""low""",0.153872,0.331015,662.029915,105140.916667,5.2570e7,0.153872,307.744795
"""25%""","""2022-02-08 00:00:00+05:30""",19.2,72.97,0.417772,null,144209.5,1028.708333,0.444393,0.513728,null,0.347003,0.417772,835.544513,160999.75,8.0499875e7,0.347003,694.00564
"""50%""","""2023-03-18 00:00:00+05:30""",28.5,76.94,0.428741,null,157871.083333,1872.666667,0.519724,0.560167,null,0.371421,0.428741,857.482495,178227.333325,8.9114e7,0.371421,742.841782
"""75%""","""2024-04-23 23:30:00+05:30""",28.7,77.14,0.438581,null,171602.083333,3205.083333,0.557195,0.588823,null,0.393326,0.438581,877.162443,194806.833333,9.7403e7,0.393326,786.651314
"""max""","""2025-05-31 23:30:00+05:30""",28.8,77.34,0.498383,null,216498.5,34155.833333,0.581367,0.630714,"""medium""",0.526451,0.498383,996.766449,249591.916667,1.2480e8,0.526451,1052.901792


In [23]:
# join the two marginal emissions dataframes
marginal_emissions_pldf = marginal_emissions_pldf.join(
    pygam_marginal_emissions,
    on=["city", "timestamp"],
    how="left",
    suffix=("pygam")
)

In [24]:
marginal_emissions_pldf

timestamp,city,land_longitude,land_latitude,ME,alpha1,alpha2,original_quantile_group_id,demand_met,quantile_marginal_emissions_tons_co2_per_MW,quantile_marginal_emissions_grams_co2_per_kWh,demand_met_MW,demand_met_kWh,land_latitudepygam,land_longitudepygam,MEpygam,demand_minus_renewables,National MW Shift,Pearson R Score,Spearman Score,Confidence Level,ME_cal,pg_marginal_emissions_tons_co2_per_MW,pg_marginal_emissions_grams_co2_per_kWh,demand_met_MWpygam,demand_met_kWhpygam,pg_marginal_emissions_tons_co2_per_MW_calibrated,pg_marginal_emissions_grams_co2_per_kWh_calibrated
"datetime[μs, Asia/Kolkata]",cat,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64
2021-01-01 00:00:00 IST,"""mumbai""",72.77,19.2,0.509737,0.805138,-0.000001,3,129481.666667,0.509737,1019.474915,129481.666667,6.4741e7,18.5,72.97,0.487644,120024.25,3102.666667,0.55606,0.587619,"""medium""",0.502544,0.487644,975.287688,129481.666667,6.4741e7,0.502544,1005.087922
2021-01-01 00:00:00 IST,"""mumbai""",72.77,19.2,0.509737,0.805138,-0.000001,3,129481.666667,0.509737,1019.474915,129481.666667,6.4741e7,18.6,72.97,0.487644,120024.25,3102.666667,0.55606,0.587619,"""medium""",0.502544,0.487644,975.287688,129481.666667,6.4741e7,0.502544,1005.087922
2021-01-01 00:00:00 IST,"""mumbai""",72.77,19.2,0.509737,0.805138,-0.000001,3,129481.666667,0.509737,1019.474915,129481.666667,6.4741e7,19.3,72.97,0.487644,120024.25,3102.666667,0.55606,0.587619,"""medium""",0.502544,0.487644,975.287688,129481.666667,6.4741e7,0.502544,1005.087922
2021-01-01 00:00:00 IST,"""mumbai""",72.77,19.2,0.509737,0.805138,-0.000001,3,129481.666667,0.509737,1019.474915,129481.666667,6.4741e7,19.1,72.87,0.487644,120024.25,3102.666667,0.55606,0.587619,"""medium""",0.502544,0.487644,975.287688,129481.666667,6.4741e7,0.502544,1005.087922
2021-01-01 00:00:00 IST,"""mumbai""",72.77,19.2,0.509737,0.805138,-0.000001,3,129481.666667,0.509737,1019.474915,129481.666667,6.4741e7,18.8,72.97,0.487644,120024.25,3102.666667,0.55606,0.587619,"""medium""",0.502544,0.487644,975.287688,129481.666667,6.4741e7,0.502544,1005.087922
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-05-31 23:30:00 IST,"""delhi""",77.34,28.8,0.295353,0.869218,-0.000001,4,212822.75,0.295353,590.706153,212822.75,1.06411375e8,28.8,76.94,0.359818,187549.25,716.5,0.407424,0.490606,"""medium""",0.217991,0.359818,719.636212,212822.75,1.06411375e8,0.217991,435.982172
2025-05-31 23:30:00 IST,"""delhi""",77.34,28.8,0.295353,0.869218,-0.000001,4,212822.75,0.295353,590.706153,212822.75,1.06411375e8,28.8,77.04,0.359818,187549.25,716.5,0.407424,0.490606,"""medium""",0.217991,0.359818,719.636212,212822.75,1.06411375e8,0.217991,435.982172
2025-05-31 23:30:00 IST,"""delhi""",77.34,28.8,0.295353,0.869218,-0.000001,4,212822.75,0.295353,590.706153,212822.75,1.06411375e8,28.8,77.14,0.359818,187549.25,716.5,0.407424,0.490606,"""medium""",0.217991,0.359818,719.636212,212822.75,1.06411375e8,0.217991,435.982172


In [26]:
# columns we want to average
avg_cols = ["demand_met_kWh", "quantile_marginal_emissions_grams_co2_per_kWh", "pg_marginal_emissions_grams_co2_per_kWh", "pg_marginal_emissions_grams_co2_per_kWh_calibrated"]

# Per-city, per-timestamp averages (ignores rows where city is null)
gapfill_marginal_emissions_pldf = (
    marginal_emissions_pldf
    .filter(pl.col("city").is_not_null())
    .group_by(["city", "timestamp"])
    .agg([pl.col(c).mean().alias(c) for c in avg_cols])
    .sort(["city", "timestamp"])
)
# `gapfill_marginal_emissions_pldf`
gapfill_marginal_emissions_pldf


city,timestamp,demand_met_kWh,quantile_marginal_emissions_grams_co2_per_kWh,pg_marginal_emissions_grams_co2_per_kWh,pg_marginal_emissions_grams_co2_per_kWh_calibrated
cat,"datetime[μs, Asia/Kolkata]",f64,f64,f64,f64
"""delhi""",2021-01-01 00:00:00 IST,6.4741e7,1026.368432,975.287688,1005.087922
"""delhi""",2021-01-01 00:30:00 IST,6.3078e7,1017.348058,990.645477,1039.275896
"""delhi""",2021-01-01 01:00:00 IST,6.2424875e7,1020.337728,972.448151,998.76683
"""delhi""",2021-01-01 01:30:00 IST,6.1435e7,1039.577621,917.840625,877.205015
"""delhi""",2021-01-01 02:00:00 IST,6.0568e7,1047.066406,868.783074,767.998002
…,…,…,…,…,…
"""mumbai""",2025-05-31 21:30:00 IST,1.0404e8,685.357381,730.640632,460.479111
"""mumbai""",2025-05-31 22:00:00 IST,1.0475e8,680.709674,726.951792,452.267384
"""mumbai""",2025-05-31 22:30:00 IST,1.0612e8,671.795602,721.221054,439.510189


In [27]:
reduced_average_emissions_pldf = reduced_average_emissions_pldf.with_columns(
    pl.col("timestamp").dt.replace_time_zone("Asia/Kolkata")
    )
reduced_average_emissions_pldf.schema

Schema([('timestamp', Datetime(time_unit='us', time_zone='Asia/Kolkata')),
        ('average_emissions_grams_co2_per_kWh', Float64)])

In [28]:
# Perform a left join to keep all rows from gapfill_marginal_emissions_pldf
city_emissions_df_complete = gapfill_marginal_emissions_pldf.join(
    reduced_average_emissions_pldf,
    on="timestamp",
    how="left"
)


In [29]:
city_emissions_df_complete.describe()

statistic,city,timestamp,demand_met_kWh,quantile_marginal_emissions_grams_co2_per_kWh,pg_marginal_emissions_grams_co2_per_kWh,pg_marginal_emissions_grams_co2_per_kWh_calibrated,average_emissions_grams_co2_per_kWh
str,str,str,f64,f64,f64,f64,f64
"""count""","""154752""","""154752""",154752.0,154752.0,154752.0,154752.0,154752.0
"""null_count""","""0""","""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",null,"""2023-03-17 23:45:00+05:30""",8.8644e7,746.193136,841.079038,706.326047,738.359913
"""std""",null,null,1.1768e7,146.336679,64.218751,142.957361,78.293185
"""min""",null,"""2021-01-01 00:00:00+05:30""",5.2570e7,207.259574,662.029915,307.744795,483.357945
"""25%""",null,"""2022-02-08 00:00:00+05:30""",8.0499875e7,653.177662,835.544513,694.00564,685.649915
"""50%""",null,"""2023-03-18 00:00:00+05:30""",8.9114e7,746.753796,857.482495,742.841782,740.288715
"""75%""",null,"""2024-04-23 23:30:00+05:30""",9.7403e7,846.129494,877.162443,786.651314,802.991573
"""max""",null,"""2025-05-31 23:30:00+05:30""",1.2480e8,1261.457496,996.766449,1052.901792,905.09159


In [30]:
city_emissions_df_complete.unique(
    subset=["city"],
    maintain_order=True
).sort(["city"]).head(5)

city,timestamp,demand_met_kWh,quantile_marginal_emissions_grams_co2_per_kWh,pg_marginal_emissions_grams_co2_per_kWh,pg_marginal_emissions_grams_co2_per_kWh_calibrated,average_emissions_grams_co2_per_kWh
cat,"datetime[μs, Asia/Kolkata]",f64,f64,f64,f64,f64
"""delhi""",2021-01-01 00:00:00 IST,6.4741e7,1026.368432,975.287688,1005.087922,802.033059
"""mumbai""",2021-01-01 00:00:00 IST,6.4741e7,1017.787187,975.287688,1005.087922,802.033059


In [31]:
trimmed_marginal_emissions_pldf = marginal_emissions_pldf.filter(
    (pl.col("timestamp") >= test_date_range_start) & (pl.col("timestamp") <= test_date_range_end)
)

In [32]:
trimmed_marginal_emissions_pldf.describe()

statistic,timestamp,city,land_longitude,land_latitude,ME,alpha1,alpha2,original_quantile_group_id,demand_met,quantile_marginal_emissions_tons_co2_per_MW,quantile_marginal_emissions_grams_co2_per_kWh,demand_met_MW,demand_met_kWh,land_latitudepygam,land_longitudepygam,MEpygam,demand_minus_renewables,National MW Shift,Pearson R Score,Spearman Score,Confidence Level,ME_cal,pg_marginal_emissions_tons_co2_per_MW,pg_marginal_emissions_grams_co2_per_kWh,demand_met_MWpygam,demand_met_kWhpygam,pg_marginal_emissions_tons_co2_per_MW_calibrated,pg_marginal_emissions_grams_co2_per_kWh_calibrated
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64
"""count""","""757125""","""757125""",757125.0,757125.0,757125.0,757125.0,757125.0,757125.0,757125.0,757125.0,757125.0,757125.0,757125.0,757125.0,757125.0,757125.0,757125.0,757125.0,757125.0,757125.0,"""757125""",757125.0,757125.0,757125.0,757125.0,757125.0,757125.0,757125.0
"""null_count""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""","""2022-05-11 00:00:00+05:30""",null,76.256667,26.676,0.353297,0.907774,-0.000002,18.393819,184546.257053,0.353297,706.593166,184546.257053,9.2273e7,26.676,76.256667,0.421628,162436.800143,1464.805335,0.448949,0.524188,null,0.355587,0.421628,843.256776,184546.257053,9.2273e7,0.355587,711.173908
"""std""",null,null,1.673881,3.851646,0.03349,0.115546,3.5255e-7,12.135526,8316.183893,0.03349,66.979681,8316.183893,4.1581e6,3.851646,1.673881,0.016072,9148.614577,1199.305723,0.085236,0.047068,null,0.035777,0.016072,32.143462,8316.183893,4.1581e6,0.035777,71.55456
"""min""","""2022-05-04 00:00:00+05:30""",null,72.77,18.5,0.268067,0.682504,-0.000002,2.0,166068.708333,0.268067,536.134053,166068.708333,8.3034e7,18.5,72.77,0.368409,140285.75,0.0,0.283472,0.448289,"""low""",0.237115,0.368409,736.817836,166068.708333,8.3034e7,0.237115,474.230186
"""25%""","""2022-05-07 12:00:00+05:30""",null,76.84,28.4,0.334347,0.805138,-0.000002,5.0,177457.416667,0.334347,668.693856,177457.416667,8.8729e7,28.4,76.84,0.418189,154453.666667,686.75,0.400718,0.487644,null,0.347931,0.418189,836.378115,177457.416667,8.8729e7,0.347931,695.861321
"""50%""","""2022-05-11 00:00:00+05:30""",null,77.04,28.5,0.359575,0.903393,-0.000002,20.0,185102.75,0.359575,719.149313,185102.75,9.2551375e7,28.5,77.04,0.424894,162361.916667,1205.958333,0.465703,0.529755,null,0.362858,0.424894,849.788949,185102.75,9.2551375e7,0.362858,725.71518
"""75%""","""2022-05-14 12:00:00+05:30""",null,77.24,28.7,0.377864,1.017534,-0.000001,30.0,191541.416667,0.377864,755.727509,191541.416667,9.5771e7,28.7,77.24,0.431823,169287.416667,1854.958333,0.518097,0.559381,null,0.37828,0.431823,863.645309,191541.416667,9.5771e7,0.37828,756.560822
"""max""","""2022-05-18 00:00:00+05:30""",null,77.34,28.8,0.426086,1.06163,-8.1658e-7,34.0,201423.666667,0.426086,852.171294,201423.666667,1.0071e8,28.8,77.34,0.443714,181606.416667,6600.5,0.579786,0.627975,"""medium""",0.404751,0.443714,887.427232,201423.666667,1.0071e8,0.404751,809.501761


In [33]:
reduced_marginal_emissions_filename = f"marginal_emissions_{str(test_date_range_start)[:10]}_to_{str(test_date_range_end)[:10]}"
reduced_marginal_emissions_filepath = os.path.join(optimisation_development_directory, reduced_marginal_emissions_filename + ".parquet")

In [34]:
try:
    trimmed_marginal_emissions_pldf.write_parquet(reduced_marginal_emissions_filepath,
                                                  compression="snappy",
                                                  statistics=True)
    print(f"Successfully saved trimmed marginal emissions data to {reduced_marginal_emissions_filepath}")

except Exception as e:
    print(f"Error occurred while saving trimmed marginal emissions data: {e}")

Successfully saved trimmed marginal emissions data to data/optimisation_development/marginal_emissions_2022-05-04_to_2022-05-18.parquet


#### Meter Data 

In [35]:
meter_readings_2022_Q2_filename = "meter_readings_2022_Q2_20250714_2324_formatted"
meter_readings_2022_Q2_filepath = os.path.join(meter_save_directory, meter_readings_2022_Q2_filename + ".parquet")

meter_readings_2022_Q2_pldf = pl.read_parquet(meter_readings_2022_Q2_filepath)

print("-"*120)
print("Meter Readings 2022 Q2\n" + "-"*120)
print("Schema:\n" + "-"*80)
display(meter_readings_2022_Q2_pldf.schema)
print("\nDescriptive Statistics:\n" + "-"*80)
display(meter_readings_2022_Q2_pldf.describe())

------------------------------------------------------------------------------------------------------------------------
Meter Readings 2022 Q2
------------------------------------------------------------------------------------------------------------------------
Schema:
--------------------------------------------------------------------------------


sys:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance


Schema([('ca_id', String),
        ('date', Datetime(time_unit='us', time_zone=None)),
        ('value', Float64),
        ('city', Categorical(ordering='physical'))])


Descriptive Statistics:
--------------------------------------------------------------------------------


statistic,ca_id,date,value,city
str,str,str,f64,str
"""count""","""346180042""","""346180042""",3.46180042e8,"""346180042"""
"""null_count""","""0""","""0""",0.0,"""0"""
"""mean""",null,"""2022-05-05 13:56:40.121286""",0.328427,null
"""std""",null,null,0.52113,null
"""min""","""60000005516""","""2022-04-01 00:00:00""",0.0,null
"""25%""",null,"""2022-04-25 08:30:00""",0.065,null
"""50%""",null,"""2022-05-05 15:00:00""",0.157,null
"""75%""",null,"""2022-05-15 21:30:00""",0.376,null
"""max""","""60029920067""","""2022-06-19 04:30:00""",94.471,null


In [36]:
# Assign time Zone
meter_readings_2022_Q2_pldf = meter_readings_2022_Q2_pldf.with_columns(
    pl.col("date").dt.replace_time_zone("Asia/Kolkata")
)
print("Updated Schema:\n" + "-"*80)
display(meter_readings_2022_Q2_pldf.schema)

Updated Schema:
--------------------------------------------------------------------------------


Schema([('ca_id', String),
        ('date', Datetime(time_unit='us', time_zone='Asia/Kolkata')),
        ('value', Float64),
        ('city', Categorical(ordering='physical'))])

In [37]:
trimmed_meter_readings_pldf = meter_readings_2022_Q2_pldf.filter(
    (pl.col("date") >= test_date_range_start) & (pl.col("date") <= test_date_range_end)
)

In [38]:
reduced_meter_readings_filename = f"meter_readings_{str(test_date_range_start)[:10]}_to_{str(test_date_range_end)[:10]}"
reduced_meter_readings_filepath = os.path.join(optimisation_development_directory, reduced_meter_readings_filename + ".parquet")

In [39]:
try:
    trimmed_meter_readings_pldf.write_parquet(reduced_meter_readings_filepath,
                                                  compression="snappy",
                                                  statistics=True)
    print(f"Successfully saved trimmed meter readings data to {reduced_meter_readings_filepath}")

except Exception as e:
    print(f"Error occurred while saving trimmed meter readings data: {e}")

Successfully saved trimmed meter readings data to data/optimisation_development/meter_readings_2022-05-04_to_2022-05-18.parquet


In [40]:
# Unique customer IDs
unique_customer_ids = trimmed_meter_readings_pldf["ca_id"].unique()
print(f"Unique customer IDs in [trimmed_meter_readings_pldf]: {unique_customer_ids}")

Unique customer IDs in [trimmed_meter_readings_pldf]: shape: (179_434,)
Series: 'ca_id' [str]
[
	"60027007511"
	"60020821298"
	"60020932277"
	"60010179558"
	"60025627849"
	…
	"60026324727"
	"60020449835"
	"60000153852"
	"60018710966"
	"60025935341"
]


### Loading and Inspecting Data

In [41]:
customers_pldf = pl.read_parquet(full_customers_filepath)

In [42]:
print("-"  *120)
print("Schema of Customers DataFrame\n"+ "-"  *120)
display(customers_pldf.schema)
print("\nSchema of Meter Readings DataFrame\n"+ "-"  *120)
display(trimmed_meter_readings_pldf.schema)
print("\nSchema of Marginal Emissions DataFrame\n"+ "-"  *120)
display(trimmed_marginal_emissions_pldf.schema)
print("\nSchema of Average Emissions DataFrame\n"+ "-"  *120)
display(trimmed_average_emissions_pldf.schema)

------------------------------------------------------------------------------------------------------------------------
Schema of Customers DataFrame
------------------------------------------------------------------------------------------------------------------------


Schema([('id', String), ('location', String), ('city', String)])


Schema of Meter Readings DataFrame
------------------------------------------------------------------------------------------------------------------------


Schema([('ca_id', String),
        ('date', Datetime(time_unit='us', time_zone='Asia/Kolkata')),
        ('value', Float64),
        ('city', Categorical(ordering='physical'))])


Schema of Marginal Emissions DataFrame
------------------------------------------------------------------------------------------------------------------------


Schema([('timestamp', Datetime(time_unit='us', time_zone='Asia/Kolkata')),
        ('city', Categorical(ordering='physical')),
        ('land_longitude', Float64),
        ('land_latitude', Float64),
        ('ME', Float64),
        ('alpha1', Float64),
        ('alpha2', Float64),
        ('original_quantile_group_id', Int64),
        ('demand_met', Float64),
        ('quantile_marginal_emissions_tons_co2_per_MW', Float64),
        ('quantile_marginal_emissions_grams_co2_per_kWh', Float64),
        ('demand_met_MW', Float64),
        ('demand_met_kWh', Float64),
        ('land_latitudepygam', Float64),
        ('land_longitudepygam', Float64),
        ('MEpygam', Float64),
        ('demand_minus_renewables', Float64),
        ('National MW Shift', Float64),
        ('Pearson R Score', Float64),
        ('Spearman Score', Float64),
        ('Confidence Level', String),
        ('ME_cal', Float64),
        ('pg_marginal_emissions_tons_co2_per_MW', Float64),
        ('pg_marginal_emissio


Schema of Average Emissions DataFrame
------------------------------------------------------------------------------------------------------------------------


Schema([('timestamp', Datetime(time_unit='us', time_zone='Asia/Kolkata')),
        ('average_emissions_grams_co2_per_kWh', Float64)])

## Joining Data

#### Average <-> Marginal Emissions

In [43]:
joined_marginal_emissions_filename = f"marginal_and_average_emissions_{str(test_date_range_start)[:10]}_to_{str(test_date_range_end)[:10]}"
joined_marginal_emissions_filepath = os.path.join(optimisation_development_directory, joined_marginal_emissions_filename + ".parquet")

In [45]:
trimmed_avg_emissions_lazy = pl.scan_parquet(trimmed_average_emissions_filepath)
trimmed_me_lazy = pl.scan_parquet(reduced_marginal_emissions_filepath)

joined_data = (
    trimmed_me_lazy
    .join(
        trimmed_avg_emissions_lazy,
        left_on=["timestamp"],
        right_on=["timestamp"],
        how="left"  # Keep all marginal emissions
    )
    # Optional: Select only the columns you need
    .select([
        # Columns from avg_emissions
        "average_emissions_grams_co2_per_kWh",
        # Columns from marginal emissions data
        "timestamp", "city", "land_longitude",
        "land_latitude", "demand_met_kWh",
        "quantile_marginal_emissions_grams_co2_per_kWh",
        "pg_marginal_emissions_grams_co2_per_kWh",
        "pg_marginal_emissions_grams_co2_per_kWh_calibrated",
        "National MW Shift",
        "Confidence Level",
        "Spearman Score",
        "Pearson R Score",
    ])
)

# Write the result to a new Parquet file
try:
    (
        joined_data
        .sink_parquet(
            joined_marginal_emissions_filepath,
            # Optional optimization parameters:
            compression="snappy",  # Good balance of speed and compression
            statistics=True,    # Write statistics for better query performance
            row_group_size=100_000  # Adjust based on your data size
        )
    )
    print(f"Successfully saved [joined_data] to {joined_marginal_emissions_filepath}")
except Exception as e:
    print(f"Error saving [joined_data] to {joined_marginal_emissions_filepath}: {e}")


Successfully saved [joined_data] to data/optimisation_development/marginal_and_average_emissions_2022-05-04_to_2022-05-18.parquet


#### Customers <-> Emissions

In [46]:
customers_coords_lzdf = (
    pl.scan_parquet(full_customers_filepath)
        .with_columns(
            # convert location fields to latitude and longitude
            pl.col("location").map_elements(wkb_to_coords,
                                                 return_dtype=pl.List(pl.Float64))
                                   .alias("location_coords"))
        .drop([
                "location"
        ])
        .with_columns([
                pl.col("location_coords").list.get(0).alias("customer_longitude"),
                pl.col("location_coords").list.get(1).alias("customer_latitude"),
            ])
        .drop(["location_coords"])
)
customers_coords_pldf = customers_coords_lzdf.collect()

In [47]:
customers_coords_trimmed_pldf = customers_coords_pldf.filter(
    pl.col("id").is_in(unique_customer_ids)
)

/var/folders/lg/rt7jvg4x71vd57p0zm_718200000gn/T/ipykernel_57282/1594750012.py:1: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  customers_coords_trimmed_pldf = customers_coords_pldf.filter(


In [41]:
customers_coords_trimmed_pldf.describe()

statistic,id,city,customer_longitude,customer_latitude
str,str,str,f64,f64
"""count""","""178377""","""178377""",178377.0,178377.0
"""null_count""","""0""","""0""",0.0,0.0
"""mean""",null,null,77.291167,28.759756
"""std""",null,null,3.263387,1.216211
"""min""","""60000005516""","""delhi""",76.954941,28.58
"""25%""",null,null,77.12768,28.688152
"""50%""",null,null,77.151595,28.703691
"""75%""",null,null,77.193455,28.723992
"""max""","""60029920067""","""delhi""",231.484813,86.320485


In [48]:
joined_marginal_emissions_pldf = pl.read_parquet(joined_marginal_emissions_filepath)

In [49]:
# Convert to GeoDataFrames for spatial operations
# And reproject to Web Mercator - Pseudo-Mercato EPSG:3857
# Uses meters as the unit of measurement - good for distance calculations

customers_data_gdf = gpd.GeoDataFrame(
    customers_coords_trimmed_pldf.to_pandas(),
    geometry=gpd.points_from_xy(
        customers_coords_trimmed_pldf["customer_longitude"],
        customers_coords_trimmed_pldf["customer_latitude"]
    ),
    crs="EPSG:3857"
)

trimmed_joined_emissions_gdf = gpd.GeoDataFrame(
    joined_marginal_emissions_pldf.to_pandas(),
    geometry=gpd.points_from_xy(
        joined_marginal_emissions_pldf["land_longitude"],
        joined_marginal_emissions_pldf["land_latitude"]
    ),
    crs="EPSG:3857"
)

In [ ]:
# mapping the nearest points from the era5 dataset to the weather dataset
customers_with_emissions_gdf = customers_data_gdf.sjoin_nearest(
    trimmed_joined_emissions_gdf,
    how="left",
    lsuffix="_customer",
    rsuffix="_me",
    distance_col="distance_between_locations_meters",
    max_distance=None,  # Set to None to find all nearest points
)

In [ ]:
print("\n" + "-" * 120)
print("Marginal Emissions data locations mapped to customer locations [customers_with_emissions_gdf]:\n" + "-" * 120)
print(f"\tShape : {customers_with_emissions_gdf.shape}")
print(f"\tType : {type(customers_with_emissions_gdf)}")
print(f"\tColumns : {customers_with_emissions_gdf.columns.tolist()}")
print("\n" + "-" * 80)
print(f"Sample rows of [customers_with_emissions_gdf]:\n" + "-" * 80)
display(customers_with_emissions_gdf.sample(8))
print("\n" + "-" * 80)


------------------------------------------------------------------------------------------------------------------------
Marginal Emissions data locations mapped to customer locations [customers_with_emissions_gdf]:
------------------------------------------------------------------------------------------------------------------------
	Shape : (120047721, 15)
	Type : <class 'geopandas.geodataframe.GeoDataFrame'>
	Columns : ['id', 'city__customer', 'customer_longitude', 'customer_latitude', 'geometry', 'index__me', 'average_emissions_grams_co2_per_kWh', 'timestamp', 'city__me', 'land_longitude', 'land_latitude', 'demand_met_kWh', 'marginal_emissions_grams_co2_per_kWh', 'ME', 'distance_between_locations_meters']

--------------------------------------------------------------------------------
Sample rows of [customers_with_emissions_gdf]:
--------------------------------------------------------------------------------


,id,city__customer,customer_longitude,customer_latitude,geometry,index__me,average_emissions_grams_co2_per_kWh,timestamp,city__me,land_longitude,land_latitude,demand_met_kWh,marginal_emissions_grams_co2_per_kWh,ME,distance_between_locations_meters
96251,60018640908,delhi,77.228741,28.701926,POINT (77.229 28.702),4093,737.513370,2022-05-16 01:30:00+05:30,delhi,77.24,28.7,9.135450e+07,672.002454,0.336001,0.011422
80074,60015603966,delhi,77.211600,28.713765,POINT (77.212 28.714),2790,756.863086,2022-05-12 05:30:00+05:30,delhi,77.24,28.7,8.933808e+07,736.833677,0.368417,0.031560
160783,60028007957,delhi,77.157653,28.715079,POINT (77.158 28.715),1239,787.927575,2022-05-09 01:30:00+05:30,delhi,77.14,28.7,9.081179e+07,677.008089,0.338504,0.023217
61323,60011137563,delhi,77.093028,28.718668,POINT (77.093 28.719),1889,681.113146,2022-05-10 08:30:00+05:30,delhi,77.14,28.7,9.710500e+07,622.327007,0.311164,0.050546
84966,60016642740,delhi,77.196145,28.754519,POINT (77.196 28.755),4733,681.113146,2022-05-10 00:00:00+05:30,delhi,77.24,28.8,9.710500e+07,680.015886,0.340008,0.063180
74698,60014286318,delhi,77.166533,28.645892,POINT (77.167 28.646),2035,740.912573,2022-05-10 18:00:00+05:30,delhi,77.14,28.6,8.957938e+07,754.393283,0.377197,0.053010
46281,60008321402,delhi,77.100242,28.691846,POINT (77.1 28.692),1000,681.113146,2022-05-09 19:30:00+05:30,delhi,77.14,28.7,9.710500e+07,730.655314,0.365328,0.040586
126666,60024295549,delhi,77.175390,28.730340,POINT (77.175 28.73),1889,681.113146,2022-05-10 08:30:00+05:30,delhi,77.14,28.7,9.710500e+07,622.327007,0.311164,0.046615



--------------------------------------------------------------------------------


In [ ]:
print(f"Descriptive statistics of [customers_with_emissions_gdf]:\n"+ "-" * 80)
display(customers_with_emissions_gdf.describe())

Descriptive statistics of [customers_with_emissions_gdf]:
--------------------------------------------------------------------------------


,customer_longitude,customer_latitude,index__me,average_emissions_grams_co2_per_kWh,land_longitude,land_latitude,demand_met_kWh,marginal_emissions_grams_co2_per_kWh,ME,distance_between_locations_meters
count,1.200477e+08,1.200477e+08,1.200477e+08,1.200477e+08,1.200477e+08,1.200477e+08,1.200477e+08,1.200477e+08,1.200477e+08,1.200477e+08
mean,7.729117e+01,2.875976e+01,5.479488e+03,7.325968e+02,7.715854e+01,2.870859e+01,9.227313e+07,7.082528e+02,3.541264e-01,1.798079e-01
std,3.263378e+00,1.216207e+00,5.304981e+03,4.537175e+01,6.058001e-02,3.473260e-02,4.158089e+06,6.624358e+01,3.312179e-02,3.472135e+00
min,7.695494e+01,2.858000e+01,1.100000e+02,6.247115e+02,7.694000e+01,2.860000e+01,8.303435e+07,5.361341e+02,2.680670e-01,1.110477e-04
25%,7.712768e+01,2.868815e+01,1.620000e+03,6.957991e+02,7.714000e+01,2.870000e+01,8.872871e+07,6.730559e+02,3.365279e-01,1.837463e-02
50%,7.715159e+01,2.870369e+01,3.201000e+03,7.384685e+02,7.714000e+01,2.870000e+01,9.255138e+07,7.208703e+02,3.604351e-01,3.222427e-02
75%,7.719345e+01,2.872399e+01,1.050100e+04,7.648485e+02,7.724000e+01,2.870000e+01,9.577071e+07,7.566462e+02,3.783231e-01,4.682137e-02
max,2.314848e+02,8.632049e+01,3.021000e+04,8.270834e+02,7.734000e+01,2.880000e+01,1.007118e+08,8.359399e+02,4.179699e-01,1.645273e+02


In [ ]:
customers_with_emissions_gdf.columns.tolist()

['id',
 'city__customer',
 'customer_longitude',
 'customer_latitude',
 'geometry',
 'index__me',
 'average_emissions_grams_co2_per_kWh',
 'timestamp',
 'city__me',
 'land_longitude',
 'land_latitude',
 'demand_met_kWh',
 'marginal_emissions_grams_co2_per_kWh',
 'ME',
 'distance_between_locations_meters']

In [ ]:
# Drop unnecessary columns
customers_with_emissions_pd = customers_with_emissions_gdf.drop(
    columns=[f"index__me", "city__me", "geometry", "land_longitude", "land_latitude",
              "distance_between_locations_meters"])

print("-"*120)
print(f"[customers_with_emissions_pd] after dropping columns:\n" + "-"*120)
print(f"\tShape : {customers_with_emissions_pd.shape}")
print(f"\tType : {type(customers_with_emissions_pd)}")
print(f"\tColumns : {customers_with_emissions_pd.columns.tolist()}")


------------------------------------------------------------------------------------------------------------------------
[customers_with_emissions_pd] after dropping columns:
------------------------------------------------------------------------------------------------------------------------
	Shape : (120047721, 9)
	Type : <class 'pandas.core.frame.DataFrame'>
	Columns : ['id', 'city__customer', 'customer_longitude', 'customer_latitude', 'average_emissions_grams_co2_per_kWh', 'timestamp', 'demand_met_kWh', 'marginal_emissions_grams_co2_per_kWh', 'ME']


In [49]:
# Reset the index of the DataFrame to ensure it is clean
customers_with_emissions_pd = customers_with_emissions_pd.reset_index(drop=True)

# Convert the pandas DataFrame to a Polars DataFrame
customers_with_emissions_pldf = pl.from_pandas(customers_with_emissions_pd)

print("-"*120)
print(f"[customers_with_emissions_pldf] after conversion: \n" + "-"*120)
print(f"Shape :\t{customers_with_emissions_pldf.shape}")
print(f"Type :\t{type(customers_with_emissions_pldf)}")
print(f"Columns :\t{customers_with_emissions_pldf.columns}")

print("\n" + "-"*80)
print(f"Sample rows of [customers_with_emissions_pldf]:\n" + "-"*80)
display(customers_with_emissions_pldf.head(5))

------------------------------------------------------------------------------------------------------------------------
[customers_with_emissions_pldf] after conversion: 
------------------------------------------------------------------------------------------------------------------------
Shape :	(120047721, 9)
Type :	<class 'polars.dataframe.frame.DataFrame'>
Columns :	['id', 'city__customer', 'customer_longitude', 'customer_latitude', 'average_emissions_grams_co2_per_kWh', 'timestamp', 'demand_met_kWh', 'marginal_emissions_grams_co2_per_kWh', 'ME']

--------------------------------------------------------------------------------
Sample rows of [customers_with_emissions_pldf]:
--------------------------------------------------------------------------------


id,city__customer,customer_longitude,customer_latitude,average_emissions_grams_co2_per_kWh,timestamp,demand_met_kWh,marginal_emissions_grams_co2_per_kWh,ME
str,str,f64,f64,f64,"datetime[μs, Asia/Kolkata]",f64,f64,f64
"""60000005516""","""delhi""",77.124072,28.728825,727.45292,2022-05-12 18:30:00 IST,8.9997e7,750.267451,0.375134
"""60000005516""","""delhi""",77.124072,28.728825,716.195122,2022-05-07 15:30:00 IST,9.9475e7,665.523966,0.332762
"""60000005516""","""delhi""",77.124072,28.728825,712.229102,2022-05-17 16:30:00 IST,9.7496e7,676.149488,0.338075
"""60000005516""","""delhi""",77.124072,28.728825,747.964818,2022-05-17 22:30:00 IST,9.5388e7,709.606799,0.354803
"""60000005516""","""delhi""",77.124072,28.728825,709.32133,2022-05-05 10:30:00 IST,9.0700875e7,716.742212,0.358371


In [50]:
display(customers_with_emissions_pldf.describe())

statistic,id,city__customer,customer_longitude,customer_latitude,average_emissions_grams_co2_per_kWh,timestamp,demand_met_kWh,marginal_emissions_grams_co2_per_kWh,ME
str,str,str,f64,f64,f64,str,f64,f64,f64
"""count""","""120047721""","""120047721""",1.20047721e8,1.20047721e8,1.20047721e8,"""120047721""",1.20047721e8,1.20047721e8,1.20047721e8
"""null_count""","""0""","""0""",0.0,0.0,0.0,"""0""",0.0,0.0,0.0
"""mean""",null,null,77.291167,28.759756,732.596824,"""2022-05-11 00:00:00+05:30""",9.2273e7,708.252794,0.354126
"""std""",null,null,3.263378,1.216207,45.371753,null,4.1581e6,66.243578,0.033122
"""min""","""60000005516""","""delhi""",76.954941,28.58,624.711512,"""2022-05-04 00:00:00+05:30""",8.3034e7,536.134053,0.268067
"""25%""",null,null,77.12768,28.688152,695.799078,"""2022-05-07 12:00:00+05:30""",8.8729e7,673.055855,0.336528
"""50%""",null,null,77.151595,28.703691,738.468516,"""2022-05-11 00:00:00+05:30""",9.2551375e7,720.870299,0.360435
"""75%""",null,null,77.193455,28.723992,764.848519,"""2022-05-14 12:00:00+05:30""",9.5771e7,756.646245,0.378323
"""max""","""60029920067""","""delhi""",231.484813,86.320485,827.083443,"""2022-05-18 00:00:00+05:30""",1.0071e8,835.939878,0.41797


In [52]:
# Rename city column
customers_with_emissions_pldf = customers_with_emissions_pldf.drop( "ME")
customers_with_emissions_pldf = customers_with_emissions_pldf.rename({"city__customer": "city"})


print("Columns: ", customers_with_emissions_pldf.columns)


Columns:  ['id', 'city', 'customer_longitude', 'customer_latitude', 'average_emissions_grams_co2_per_kWh', 'timestamp', 'demand_met_kWh', 'marginal_emissions_grams_co2_per_kWh']


In [53]:
# reorder columns
customers_with_emissions_pldf = customers_with_emissions_pldf.select([
   "timestamp", "id", "city", "customer_longitude",
    "customer_latitude", "demand_met_kWh",
    "average_emissions_grams_co2_per_kWh",
    "marginal_emissions_grams_co2_per_kWh"
])
print("Columns: ", customers_with_emissions_pldf.columns)


Columns:  ['timestamp', 'id', 'city', 'customer_longitude', 'customer_latitude', 'demand_met_kWh', 'average_emissions_grams_co2_per_kWh', 'marginal_emissions_grams_co2_per_kWh']


In [55]:
customers_with_emissions_filename = "customers_ids_with_emissions"
customers_with_emissions_filepath = os.path.join(optimisation_development_directory, customers_with_emissions_filename + ".parquet")

In [56]:
try:
    customers_with_emissions_pldf.write_parquet(customers_with_emissions_filepath,
                                                         compression="snappy",
                                                         statistics=True)
    print(f"Successfully saved [customers_with_emissions_pldf] to {customers_with_emissions_filepath}")

except Exception as e:
    print(f"Error saving [customers_with_emissions_pldf] to {customers_with_emissions_filepath}: {e}")

Successfully saved [customers_with_emissions_pldf] to data/optimisation_development/customers_ids_with_emissions.parquet


#### Meter Readings <-> Emissions

In [58]:
customers_with_emissions_pldf.schema

Schema([('timestamp', Datetime(time_unit='us', time_zone='Asia/Kolkata')),
        ('id', String),
        ('city', String),
        ('customer_longitude', Float64),
        ('customer_latitude', Float64),
        ('demand_met_kWh', Float64),
        ('average_emissions_grams_co2_per_kWh', Float64),
        ('marginal_emissions_grams_co2_per_kWh', Float64)])

In [59]:
trimmed_meter_readings_pldf.schema

Schema([('ca_id', String),
        ('date', Datetime(time_unit='us', time_zone='Asia/Kolkata')),
        ('value', Float64),
        ('city', Categorical(ordering='physical'))])

In [60]:
customers_with_emissions_pldf.describe()

statistic,timestamp,id,city,customer_longitude,customer_latitude,demand_met_kWh,average_emissions_grams_co2_per_kWh,marginal_emissions_grams_co2_per_kWh
str,str,str,str,f64,f64,f64,f64,f64
"""count""","""120047721""","""120047721""","""120047721""",1.20047721e8,1.20047721e8,1.20047721e8,1.20047721e8,1.20047721e8
"""null_count""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0
"""mean""","""2022-05-11 00:00:00+05:30""",null,null,77.291167,28.759756,9.2273e7,732.596824,708.252794
"""std""",null,null,null,3.263378,1.216207,4.1581e6,45.371753,66.243578
"""min""","""2022-05-04 00:00:00+05:30""","""60000005516""","""delhi""",76.954941,28.58,8.3034e7,624.711512,536.134053
"""25%""","""2022-05-07 12:00:00+05:30""",null,null,77.12768,28.688152,8.8729e7,695.799078,673.055855
"""50%""","""2022-05-11 00:00:00+05:30""",null,null,77.151595,28.703691,9.2551375e7,738.468516,720.870299
"""75%""","""2022-05-14 12:00:00+05:30""",null,null,77.193455,28.723992,9.5771e7,764.848519,756.646245
"""max""","""2022-05-18 00:00:00+05:30""","""60029920067""","""delhi""",231.484813,86.320485,1.0071e8,827.083443,835.939878


In [61]:
trimmed_meter_readings_pldf.describe()

statistic,ca_id,date,value,city
str,str,str,f64,str
"""count""","""118643003""","""118643003""",1.18643003e8,"""118643003"""
"""null_count""","""0""","""0""",0.0,"""0"""
"""mean""",null,"""2022-05-10 23:31:16.797125+05:…",0.349623,null
"""std""",null,null,0.541503,null
"""min""","""60000005516""","""2022-05-04 00:00:00+05:30""",0.0,null
"""25%""",null,"""2022-05-07 11:30:00+05:30""",0.069,null
"""50%""",null,"""2022-05-10 23:00:00+05:30""",0.166,null
"""75%""",null,"""2022-05-14 13:30:00+05:30""",0.417,null
"""max""","""60029920067""","""2022-05-18 00:00:00+05:30""",49.438,null


In [62]:
joined_meter_readings_marginal_emissions_filename = reduced_meter_readings_filename + "_with_marginal_emissions"
joined_meter_readings_marginal_emissions_filepath = os.path.join(optimisation_development_directory, joined_meter_readings_marginal_emissions_filename + ".parquet")

In [63]:
trimmed_meter_lazy = pl.scan_parquet(reduced_meter_readings_filepath)
trimmed_customers_me_lazy = pl.scan_parquet(customers_with_emissions_filepath)

joined_data = (
    trimmed_meter_lazy
    .join(
        trimmed_customers_me_lazy,
        left_on=["ca_id", "date"],
        right_on=["id" ,"timestamp"],
        how="left"  # Keep all meter readings
    )
    # Optional: Select only the columns you need
    .select([
        # Columns from meter readings
        "ca_id", "date", "value", "city",
        # Columns from customer data
        "customer_longitude", "customer_latitude",
        "average_emissions_grams_co2_per_kWh", "demand_met_kWh",
        "marginal_emissions_grams_co2_per_kWh"
    ])
)

# Write the result to a new Parquet file
try:
    (
        joined_data
        .sink_parquet(
            joined_meter_readings_marginal_emissions_filepath,
            # Optional optimization parameters:
            compression="snappy",  # Good balance of speed and compression
            statistics=True,    # Write statistics for better query performance
            row_group_size=100_000  # Adjust based on your data size
        )
    )
    print(f"Successfully saved [joined_data] to {joined_meter_readings_marginal_emissions_filepath}")
except Exception as e:
    print(f"Error saving [joined_data] to {joined_meter_readings_marginal_emissions_filepath}: {e}")


Successfully saved [joined_data] to data/optimisation_development/meter_readings_2022-05-04_to_2022-05-18_with_marginal_emissions.parquet


In [64]:
test_df = pl.read_parquet(joined_meter_readings_marginal_emissions_filepath)
test_df.schema

Schema([('ca_id', String),
        ('date', Datetime(time_unit='us', time_zone='Asia/Kolkata')),
        ('value', Float64),
        ('city', Categorical(ordering='physical')),
        ('customer_longitude', Float64),
        ('customer_latitude', Float64),
        ('average_emissions_grams_co2_per_kWh', Float64),
        ('demand_met_kWh', Float64),
        ('marginal_emissions_grams_co2_per_kWh', Float64)])

In [65]:
display(test_df.head(5))

ca_id,date,value,city,customer_longitude,customer_latitude,average_emissions_grams_co2_per_kWh,demand_met_kWh,marginal_emissions_grams_co2_per_kWh
str,"datetime[μs, Asia/Kolkata]",f64,cat,f64,f64,f64,f64,f64
"""60009582119""",2022-05-04 01:30:00 IST,0.15,"""delhi""",77.14938,28.68702,783.035512,9.0954e7,646.189219
"""60021047877""",2022-05-04 01:30:00 IST,0.09,"""delhi""",77.204947,28.676119,783.035512,9.0954e7,646.189219
"""60022368124""",2022-05-04 01:30:00 IST,0.365,"""delhi""",77.169541,28.728754,783.035512,9.0954e7,646.189219
"""60025808050""",2022-05-04 01:30:00 IST,0.457,"""delhi""",77.13155,28.699188,783.035512,9.0954e7,646.189219
"""60010122004""",2022-05-04 01:30:00 IST,0.25,"""delhi""",77.190182,28.76294,783.035512,9.0954e7,646.189219


In [66]:
test_df.describe()

statistic,ca_id,date,value,city,customer_longitude,customer_latitude,average_emissions_grams_co2_per_kWh,demand_met_kWh,marginal_emissions_grams_co2_per_kWh
str,str,str,f64,str,f64,f64,f64,f64,f64
"""count""","""118643003""","""118643003""",1.18643003e8,"""118643003""",1.17974327e8,1.17974327e8,1.17974327e8,1.17974327e8,1.17974327e8
"""null_count""","""0""","""0""",0.0,"""0""",668676.0,668676.0,668676.0,668676.0,668676.0
"""mean""",null,"""2022-05-10 23:31:16.797125+05:…",0.349623,null,77.291045,28.759742,733.243959,9.2286e7,708.512638
"""std""",null,null,0.541503,null,3.262976,1.216057,44.958256,4.1689e6,66.360743
"""min""","""60000005516""","""2022-05-04 00:00:00+05:30""",0.0,null,76.954941,28.58,624.711512,8.3034e7,536.134053
"""25%""",null,"""2022-05-07 11:30:00+05:30""",0.069,null,77.12764,28.688147,696.592516,8.8729e7,673.338588
"""50%""",null,"""2022-05-10 23:00:00+05:30""",0.166,null,77.15129,28.703647,738.809205,9.2563e7,720.94116
"""75%""",null,"""2022-05-14 13:30:00+05:30""",0.417,null,77.193402,28.723953,764.851089,9.5808e7,756.948061
"""max""","""60029920067""","""2022-05-18 00:00:00+05:30""",49.438,null,231.484813,86.320485,827.083443,1.0071e8,835.939878


In [ ]:
filled_df = (test_df.join(
        city_emissions_df_complete,
        left_on=["date", "city"],
        right_on=["timestamp", "city"],
        how="left"
  ).with_columns([
        # Fill null demand_met_kWh with the averaged version
        pl.when(pl.col("demand_met_kWh").is_null())
          .then(pl.col("demand_met_kWh_right"))  # from joined table
          .otherwise(pl.col("demand_met_kWh"))
          .alias("demand_met_kWh"),

        # Fill null marginal_emissions with the averaged version
        pl.when(pl.col("marginal_emissions_grams_co2_per_kWh").is_null())
          .then(pl.col("marginal_emissions_grams_co2_per_kWh_right"))
          .otherwise(pl.col("marginal_emissions_grams_co2_per_kWh"))
          .alias("marginal_emissions_grams_co2_per_kWh"),

        # Similarly for average emissions if needed
        pl.when(pl.col("average_emissions_grams_co2_per_kWh").is_null())
          .then(pl.col("average_emissions_grams_co2_per_kWh_right"))
          .otherwise(pl.col("average_emissions_grams_co2_per_kWh"))
          .alias("average_emissions_grams_co2_per_kWh")
    ])
    # Drop the temporary columns from the join
    .drop(["demand_met_kWh_right", "marginal_emissions_grams_co2_per_kWh_right", "average_emissions_grams_co2_per_kWh_right"])
)

In [72]:
# reorder
filled_df = filled_df.select([
    "ca_id",
    "date",
    "city",
    "customer_longitude",
    "customer_latitude",
    "value",
    "demand_met_kWh",
    "marginal_emissions_grams_co2_per_kWh",
    "average_emissions_grams_co2_per_kWh"
])

In [73]:
filled_df.describe(())

statistic,ca_id,date,city,customer_longitude,customer_latitude,value,demand_met_kWh,marginal_emissions_grams_co2_per_kWh,average_emissions_grams_co2_per_kWh
str,str,str,str,f64,f64,f64,f64,f64,f64
"""count""","""118643003""","""118643003""","""118643003""",1.17974327e8,1.17974327e8,1.18643003e8,1.18643003e8,1.18643003e8,1.18643003e8
"""null_count""","""0""","""0""","""0""",668676.0,668676.0,0.0,0.0,0.0,0.0
"""mean""",null,"""2022-05-10 23:31:16.797125+05:…",null,77.291045,28.759742,0.349623,9.2286e7,708.509083,733.241389
"""std""",null,null,null,3.262976,1.216057,0.541503,4.1689e6,66.359649,44.958014
"""min""","""60000005516""","""2022-05-04 00:00:00+05:30""",null,76.954941,28.58,0.0,8.3034e7,536.134053,624.711512
"""max""","""60029920067""","""2022-05-18 00:00:00+05:30""",null,231.484813,86.320485,49.438,1.0071e8,835.939878,827.083443


In [ ]:
try:
    filled_df.write_parquet(joined_meter_readings_marginal_emissions_filepath,
                                                         compression="snappy",
                                                         statistics=True)
    print(f"Successfully saved [filled_df] to {joined_meter_readings_marginal_emissions_filepath}")

except Exception as e:
    print(f"Error saving [filled_df] to {joined_meter_readings_marginal_emissions_filepath}: {e}")

Successfully saved [filled_df] to data/optimisation_development/meter_readings_2022-05-04_to_2022-05-18_with_marginal_emissions.parquet


: 